In [1]:
import numpy as np
from csc.utils import *
from csc.atoms import *
from csc.dictionary import *
from csc.workbench import *
from itertools import product
import matplotlib.pyplot as plt

# Atom parameters
bmin = 0.005
bmax = 0.06
b_step = 0.005
b_values = np.arange(bmin, bmax, b_step)

ymin = 0.010
ymax = 0.036
y_step = 0.002
y_values = np.arange(ymin, ymax, y_step)

s_values = [1]

dictionary = ZSDictionary.from_values(b_values, y_values, s_values)
print('The dictionary has {} different atoms'.format(len(dictionary)))

The dictionary has 143 different atoms


In [2]:
batchSize = 200
input_signals_db = f'/home/gomes/synthetic-data/csc-signals-{batchSize}.json'
input_signals_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\\synthetic-data\\csc-signals-{batchSize}.json'

In [3]:
batchSize = 200
path_to_signals_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\\synthetic-data\\csc-signals-{batchSize}.json'
path_to_mmpdf_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\\synthetic-data\\csc-mmpdf-{batchSize}.json'
path_to_mp_db = f'C:\\Users\\gomes\\Documents\\ENS\\00_M1\\00_BORELLI\\01_WorkInProgress\\\synthetic-data\\csc-mp-{batchSize}.json'

workbench = CSCWorkbench(path_to_signals_db)
workbench.load_data()
workbench.set_dictionary(dictionary)

In [4]:
with open(path_to_signals_db, 'r') as f:
            output_data = json.load(f)
            signals_results = output_data['signals']

id0 = next(x['id'] for x in signals_results if x['sparsity'] == 6 and x['snr'] == 10)
signal_id = id0+4
signal_dict = workbench.signalDictFromId(signal_id)

In [5]:
dictionary.getAtomsLength()

492

In [6]:
print(f'Nb atoms: {len(dictionary)}')

nb_valid_offset = len(signal_dict['signal']) - dictionary.getAtomsLength() + 1
print(f'Nb valid offsets: {nb_valid_offset}')
print(f'Flatten dim = nb_atoms * nb_valid_offset = {len(dictionary)} * {nb_valid_offset} = {len(dictionary) * nb_valid_offset}')

Nb atoms: 143
Nb valid offsets: 247
Flatten dim = nb_atoms * nb_valid_offset = 143 * 247 = 35321


In [7]:
csc_result = dictionary.alphaCSCResult(signal_dict, lmbda=1.42e-1, verbose=True)

(143, 247)
Nb nnz indexes = 1


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
csc_activations = csc_result['activations']
print(csc_activations.shape)
print(csc_activations[:-3])

(35321,)
[0. 0. 0. ... 0. 0. 0.]


In [ ]:
csc_result = dictionary.alphaCSCAutoCalibrationResult(signal_dict, nb_activations=3, verbose=True)

    Calibration interval n°1 [1.00e-04, 1.00e+03] :
       0 - lmba = 1.00e-04 : nb_act = 25627
       1 - lmba = 5.99e-04 : nb_act = 25609
       2 - lmba = 3.59e-03 : nb_act = 25494
       3 - lmba = 2.15e-02 : nb_act = 24766
       4 - lmba = 1.29e-01 : nb_act = 20578
       5 - lmba = 7.74e-01 : nb_act = 7580
       6 - lmba = 4.64e+00 : nb_act = 0
       7 - lmba = 2.78e+01 : nb_act = 0
       8 - lmba = 1.67e+02 : nb_act = 0
       9 - lmba = 1.00e+03 : nb_act = 0
   >>> Last non-zero index : 5 for 7580 activations
    Calibration interval n°2 [7.74e-01, 4.64e+00] :
       0 - lmba = 7.74e-01 : nb_act = 7580
       1 - lmba = 9.45e-01 : nb_act = 5005
       2 - lmba = 1.15e+00 : nb_act = 2437
       3 - lmba = 1.41e+00 : nb_act = 734
       4 - lmba = 1.72e+00 : nb_act = 6
       5 - lmba = 2.09e+00 : nb_act = 0
       6 - lmba = 2.56e+00 : nb_act = 0
       7 - lmba = 3.12e+00 : nb_act = 0
       8 - lmba = 3.80e+00 : nb_act = 0
       9 - lmba = 4.64e+00 : nb_act = 0
   >>> Las

In [ ]:
print(csc_result.keys())
print(len(csc_result['activations']))
print(csc_result['atoms_idx'])
print(len(csc_result['atoms_idx']))
print(csc_result['positions_idx'])

dict_keys(['activations', 'atoms_idx', 'positions_idx'])
35321
[ 25  26  64 ... 105 132  67]
20578
[243 243 241 ... 115 205 117]
